## S3 Vectors - Specific, Cost Estimation Queries.

## Notation
- **N**: number of `QueryVectors` API calls.
- **K**: `topK` per call (1–30).
- **C**: vector comparisons per call (internal compute S3 Vectors charges for).
- **P**: price per 1,000,000 comparisons (assume $1.00 for planning).

**Cost (USD)** = (N × C / 1,000,000) × P

- Rules of thumb for C:
- For K in [10, 30], a good planning range is **C ≈ 300–500** comparisons/call.
- A simple linear model that works well: **C ≈ 15 × K** (midpoint).
-  Three-anchor gold suite, two calls each (N=6), K=20 → midpoint ≈ 6 × 0.000015 × 20 = $0.0018; band ≈ $0.0018–$0.0030.
- One evaluation pass of 40–50 tests, one call each (N∈{40,50}), K=20 → midpoint ≈ $0.012–$0.015.
- Development total of 1,000–2,000 calls over two months, K=30 → midpoint ≈ $0.45–$0.90; sensitivity band ≈ $0.30–$1.00. (That is N×450/1e6×$1 vs N×[300,500]/1e6×$1.)

### Explanation:
- Think in terms of how many candidate vectors S3 Vectors (or any ANN service) **must inspect per query** to find the top-K most similar ones.
- Each query inspects not just the final K results but **a larger candidate set C** (the number of actual dot-product or cosine comparisons performed). C depends on how broad the search space is.
- Filtered: A few dozen to a few hundred sentences. 300-400?
- Open: Whole index, all filings, Hundreds of thousands of sentences. 500-open?
- Hence, we give them different slopes for C. not fixed constants, just planning heuristics.
- Open query: evaluate global recall: can the system still find relevant sentences across filings, years, or companies?
- **Why does midpoint**: we don’t know the exact number of comparisons per query—S3 doesn’t expose that metric—you plan around a midpoint estimate.
- literature-based boundaries: HNSW, FAISS, and similar ANN indexes typically use 300–500 internal comparisons for topK≈10–30.
- we choose a midpoint constant (≈15×K) as a convenient linear approximation.
- **sensitivity band**: safety range, you don’t know the exact number of internal vector comparisons, depends on: pruning heuristics, query distribution, etc.
- “Even if the service internally does as few as 300 or as many as 500 comparisons per query, my total cost will still land within this range.”
- ` Cost_low  = (N_total * 300) / 1e6 * P , Cost_high = (N_total * 500) / 1e6 * P `
- band gives you context: “this cost might fluctuate by 20–30% either way depending on runtime search intensity.”


| Concept              | Meaning                               | Analogy                                        |
| -------------------- | ------------------------------------- | ---------------------------------------------- |
| **Filtered gold**    | Search constrained to same section    | “Look only inside this folder.”                |
| **Open gold**        | Search across all sections            | “Search the whole drive.”                      |
| **C (comparisons)**  | Internal similarity checks per query  | “How many files do I open to find the top 30?” |
| **α (slope)**        | Rough ratio of comparisons to topK    | “How thorough is my search?”                   |
| **Midpoint**         | A central planning value for C        | “Average workload per query.”                  |
| **Sensitivity band** | Range [C_low, C_high] for uncertainty | “Fastest vs slowest possible runs.”            |


In [2]:
# ============================================================
# S3 Vectors — Minimal Cost Estimator (N, K, C, P model)
# ============================================================

# Inputs (edit these two lines most of the time)
N = 200          # number of QueryVectors calls in the run
K = 30           # topK per call (1–30)

# Model/price assumptions (change only if pricing/rules change)
P_USD_PER_MILLION = 1.00     # $ per 1,000,000 vector comparisons (planning price)
ALPHA = 15                   # midpoint slope for C ≈ ALPHA * K

# Optional scenario helpers (uncomment and set when convenient)
# Example A: gold suite with filtered + open per anchor (2 calls each)
# anchors = 3; calls_per_anchor = 2; N = anchors * calls_per_anchor
# Example B: dev batch where you ran 50 tests, one call each
# tests = 50; N = tests * 1
# Example C: multi-month dev total (e.g., 2,000 calls)
# N = 2000

def comps_midpoint(K: int, alpha: int = ALPHA) -> int:
    """Midpoint rule for comparisons per call."""
    return alpha * K

def cost_usd(N: int, C_per_call: int, price_per_million: float = P_USD_PER_MILLION) -> float:
    """USD = (N * C) / 1e6 * P."""
    return (N * C_per_call / 1_000_000.0) * price_per_million

# Core calculations
C_mid = comps_midpoint(K)                        # midpoint comparisons/call
c_mid = cost_usd(N, C_mid)                       # midpoint cost
# Sensitivity band for common K range: 300–500 comps/call
C_low, C_high = 300, 500
c_low = cost_usd(N, C_low)
c_high = cost_usd(N, C_high)

print("="*60)
print("S3 Vectors — Minimal Cost Estimator")
print("="*60)
print(f"Inputs: N={N} calls, K={K}, P=${P_USD_PER_MILLION}/1e6 comps")
print(f"Model : C≈{ALPHA}×K → {C_mid} comps/call (midpoint)")
print("-"*60)
print(f"Midpoint estimate : ${c_mid:,.4f}")
print(f"Sensitivity (300–500 comps/call): ${c_low:,.4f} – ${c_high:,.4f}")
print("-"*60)
# Convenience: cost per call and quick formula N * 0.000015 * K
cost_per_call_mid = cost_usd(1, C_mid)
print(f"Cost/call (midpoint) : ${cost_per_call_mid:,.6f}")
print(f"Quick formula        : Cost ≈ N × 0.000015 × K  (P=$1, C≈15K)")


S3 Vectors — Minimal Cost Estimator
Inputs: N=200 calls, K=30, P=$1.0/1e6 comps
Model : C≈15×K → 450 comps/call (midpoint)
------------------------------------------------------------
Midpoint estimate : $0.0900
Sensitivity (300–500 comps/call): $0.0600 – $0.1000
------------------------------------------------------------
Cost/call (midpoint) : $0.000450
Quick formula        : Cost ≈ N × 0.000015 × K  (P=$1, C≈15K)


In [3]:
# ============================================================
# S3 Vectors — Minimal Cost Estimator (V2 for Gold Test P2)
# N–K–C–P with split regimes: filtered vs open
# ============================================================

# -----------------------------
# Inputs 
# -----------------------------
anchors = 60          # how many anchors P2 evaluated (MAX_ANCHORS)
calls_per_anchor = 2  # P2 does: filtered + open
K_filt = 30           # TOPK_FILT
K_open = 30           # TOPK_OPEN

# also want to add your “dev totals” across weeks/months:
extra_dev_calls = 0   # e.g., 1000 or 2000 to represent separate dev testing

# -----------------------------
# Pricing assumptions (planning)
# -----------------------------
P_USD_PER_MILLION = 1.00  # $ per 1,000,000 comparisons (planning price)
# Midpoint comparison models:
ALPHA_OPEN = 15           # C_open ≈ 15 * K_open   (midpoint)
ALPHA_FILT = 12           # C_filt ≈ 12 * K_filt   (prefiltering tends lower)

# Sensitivity band (per-call comparisons) for reporting
BAND_LOW  = 300
BAND_HIGH = 500

# -----------------------------
# Helpers
# -----------------------------
def comps_midpoint(K: int, alpha: int) -> int:
    return alpha * K

def cost_usd_from_calls(N_calls: int, C_per_call: int, P: float) -> float:
    # USD = (N * C) / 1e6 * P
    return (N_calls * C_per_call / 1_000_000.0) * P

# -----------------------------
# Mirror Gold Test P2 call accounting
# -----------------------------
N_filt = anchors                 # one filtered query per anchor
N_open = anchors                 # one open query per anchor
N_total = N_filt + N_open + int(extra_dev_calls)

# Per-regime comparisons (midpoints)
C_mid_filt = comps_midpoint(K_filt, ALPHA_FILT)
C_mid_open = comps_midpoint(K_open, ALPHA_OPEN)

# Costs (midpoints)
cost_filt_mid = cost_usd_from_calls(N_filt, C_mid_filt, P_USD_PER_MILLION)
cost_open_mid = cost_usd_from_calls(N_open, C_mid_open, P_USD_PER_MILLION)
cost_extra_mid = cost_usd_from_calls(extra_dev_calls, comps_midpoint(K_open, ALPHA_OPEN), P_USD_PER_MILLION)

# Sensitivity: assume the same band for both regimes when you just want bounds
cost_low  = cost_usd_from_calls(N_total, BAND_LOW,  P_USD_PER_MILLION)
cost_high = cost_usd_from_calls(N_total, BAND_HIGH, P_USD_PER_MILLION)

# Per-call convenience numbers (midpoints)
cost_per_call_filt_mid = cost_usd_from_calls(1, C_mid_filt, P_USD_PER_MILLION)
cost_per_call_open_mid = cost_usd_from_calls(1, C_mid_open, P_USD_PER_MILLION)

# -----------------------------
# Output
# -----------------------------
print("="*64)
print("S3 Vectors — Minimal Cost Estimator (V2, Gold Test P2 aware)")
print("="*64)
print(f"Anchors             : {anchors}")
print(f"Calls/anchor (P2)   : {calls_per_anchor}  (filtered + open)")
print(f"Regime K values     : K_filt={K_filt}, K_open={K_open}")
print(f"Planning price (P)  : ${P_USD_PER_MILLION:.2f} / 1,000,000 comps")
print(f"Extra dev calls     : {extra_dev_calls}")
print("-"*64)
print(f"N_filt, N_open      : {N_filt}, {N_open}  → N_total={N_total}")
print(f"C_mid (filt, open)  : {C_mid_filt}, {C_mid_open} comps/call")
print(f"Cost/call (filt)    : ${cost_per_call_filt_mid:,.6f}")
print(f"Cost/call (open)    : ${cost_per_call_open_mid:,.6f}")
print("-"*64)
print(f"Midpoint cost (filt): ${cost_filt_mid:,.4f}")
print(f"Midpoint cost (open): ${cost_open_mid:,.4f}")
print(f"Midpoint cost (extra): ${cost_extra_mid:,.4f}")
print(f"Midpoint total      : ${cost_filt_mid + cost_open_mid + cost_extra_mid:,.4f}")
print("-"*64)
print(f"Sensitivity band @ {BAND_LOW}–{BAND_HIGH} comps/call on N_total={N_total}:")
print(f"Total cost range    : ${cost_low:,.4f} – ${cost_high:,.4f}")
print("="*64)
# Quick head-math formulas you can rely on:
#  - Open midpoint:   cost ≈ N_open * 0.000015 * K_open
#  - Filtered midpt:  cost ≈ N_filt * 0.000012 * K_filt
#  - Uniform band:    replace 0.000015*K with {BAND_LOW or BAND_HIGH}/1e6


S3 Vectors — Minimal Cost Estimator (V2, Gold Test P2 aware)
Anchors             : 60
Calls/anchor (P2)   : 2  (filtered + open)
Regime K values     : K_filt=30, K_open=30
Planning price (P)  : $1.00 / 1,000,000 comps
Extra dev calls     : 0
----------------------------------------------------------------
N_filt, N_open      : 60, 60  → N_total=120
C_mid (filt, open)  : 360, 450 comps/call
Cost/call (filt)    : $0.000360
Cost/call (open)    : $0.000450
----------------------------------------------------------------
Midpoint cost (filt): $0.0216
Midpoint cost (open): $0.0270
Midpoint cost (extra): $0.0000
Midpoint total      : $0.0486
----------------------------------------------------------------
Sensitivity band @ 300–500 comps/call on N_total=120:
Total cost range    : $0.0360 – $0.0600


## Cost Estimation for S3 Vectors

- Parquet file: 360 MB (compressed, columnar format)
- Logical size for AWS billing: 796 MB (uncompressed, row format)
- Difference: 2.2x larger for billing purposes
- ** Logical size calculation:
- Vector data: 203,076 × 1024 × 4 bytes = 833 MB
- Keys: 203,076 × 20 bytes = 4 MB
- Metadata: 203,076 × 94 bytes = 19 MB
- Total logical: ~856 MB = 0.82 GB
- **sampled one row's metadata size**, differs slightly.

In [4]:
# ============================================================================
# S3 VECTORS COST ESTIMATION
# Estimates storage, upload, and query costs for your vector index
# ============================================================================

import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / 'loaders'))

from ml_config_loader import MLConfig
import polars as pl

# ============================================================================
# PARAMETERS - Adjust for your scenario
# ============================================================================

# Query volume scenarios
DEVELOPMENT_QUERIES_PER_MONTH = 1000      # Light testing
MODERATE_QUERIES_PER_MONTH = 10000        # Active development
PRODUCTION_QUERIES_PER_MONTH = 100000     # Production usage

# Choose scenario
QUERY_SCENARIO = "development"  # "development" | "moderate" | "production"

# Provider
S3VECTORS_PROVIDER = "cohere_1024d"

# ============================================================================
# AWS PRICING CONSTANTS (us-east-1, as of Nov 2025)
# ============================================================================

# Storage
STORAGE_COST_PER_GB_MONTH = 0.06  # $0.06/GB-month

# PutVectors (Upload)
PUTVECTORS_COST_PER_GB = 0.20  # $0.20/GB uploaded

# QueryVectors
QUERY_API_COST_PER_MILLION = 2.50  # $2.50 per million API calls

# Query data processing (tiered)
QUERY_DATA_COST_TIER1 = 0.004  # $/TB for first 100K vectors processed
QUERY_DATA_COST_TIER2 = 0.002  # $/TB for next 150K vectors

# ============================================================================
# LOAD STAGE 3 DATA
# ============================================================================

config = MLConfig()
cache_path = config.get_s3vectors_cache_path(S3VECTORS_PROVIDER)

if not cache_path.exists():
    print(f"⚠️  Stage 3 data not found: {cache_path}")
    print("Run BUILD_S3VECTORS_TABLE=True first")
    sys.exit(1)

df_stage3 = pl.read_parquet(cache_path)

print("="*70)
print("S3 VECTORS COST ESTIMATION")
print("="*70)
print(f"Provider: {S3VECTORS_PROVIDER}")
print(f"Data source: {cache_path.name}")

# ============================================================================
# CALCULATE LOGICAL SIZE
# ============================================================================

total_vectors = len(df_stage3)

# Size components per vector
BYTES_PER_FLOAT32 = 4
DIMENSION = config.s3vectors_dimensions(S3VECTORS_PROVIDER)
VECTOR_DATA_SIZE = DIMENSION * BYTES_PER_FLOAT32  # 1024 * 4 = 4,096 bytes

# Sample one row to estimate metadata size
sample_row = df_stage3.row(0, named=True)

# Key size (sentenceID_numsurrogate as string)
key_size = len(str(sample_row['sentenceID_numsurrogate']))

# Filterable metadata (5 fields)
filterable_size = (
    4 +  # cik_int (int32)
    2 +  # report_year (int16)
    len(str(sample_row['section_name'])) +  # varchar
    len(str(sample_row['sic'])) +  # varchar
    2    # sentence_pos (int16)
)

# Non-filterable metadata (3 fields)
nonfilterable_size = (
    len(str(sample_row['sentenceID'])) +
    len(str(sample_row['embedding_id'])) +
    4  # section_sentence_count (int32)
)

# Total per vector (for storage and upload)
total_size_per_vector = (
    VECTOR_DATA_SIZE +
    key_size +
    filterable_size +
    nonfilterable_size
)

# Query processing size (excludes non-filterable metadata)
query_processing_size_per_vector = (
    VECTOR_DATA_SIZE +
    key_size +
    filterable_size
)

total_storage_gb = (total_vectors * total_size_per_vector) / (1024 ** 3)
total_upload_gb = total_storage_gb  # Same logical size

print(f"\n[Vector Index Statistics]")
print(f"  Total vectors: {total_vectors:,}")
print(f"  Dimension: {DIMENSION}d")

print(f"\n[Size Breakdown per Vector]")
print(f"  Vector data: {VECTOR_DATA_SIZE:,} bytes ({VECTOR_DATA_SIZE/1024:.2f} KB)")
print(f"  Key: {key_size} bytes")
print(f"  Filterable metadata: {filterable_size} bytes")
print(f"  Non-filterable metadata: {nonfilterable_size} bytes")
print(f"  Total per vector: {total_size_per_vector:,} bytes ({total_size_per_vector/1024:.2f} KB)")

print(f"\n[Total Logical Size]")
print(f"  Storage size: {total_storage_gb:.4f} GB ({total_storage_gb*1024:.2f} MB)")
print(f"  Upload size: {total_upload_gb:.4f} GB")

# ============================================================================
# COST 1: ONE-TIME UPLOAD (PutVectors)
# ============================================================================

upload_cost = total_upload_gb * PUTVECTORS_COST_PER_GB

print(f"\n{'='*70}")
print(f"COST 1: ONE-TIME UPLOAD (PutVectors)")
print(f"{'='*70}")
print(f"  Data uploaded: {total_upload_gb:.4f} GB")
print(f"  Rate: ${PUTVECTORS_COST_PER_GB:.2f} per GB")
print(f"  Total upload cost: ${upload_cost:.4f}")
print(f"  Status: Pay once when inserting vectors")

# ============================================================================
# COST 2: MONTHLY STORAGE
# ============================================================================

monthly_storage_cost = total_storage_gb * STORAGE_COST_PER_GB_MONTH

print(f"\n{'='*70}")
print(f"COST 2: MONTHLY STORAGE")
print(f"{'='*70}")
print(f"  Data stored: {total_storage_gb:.4f} GB")
print(f"  Rate: ${STORAGE_COST_PER_GB_MONTH:.2f} per GB-month")
print(f"  Monthly storage cost: ${monthly_storage_cost:.4f}")
print(f"  Annual storage cost: ${monthly_storage_cost * 12:.2f}")

# ============================================================================
# COST 3: QUERY COSTS (Scenario-based)
# ============================================================================

# Map scenario to query count
query_counts = {
    "development": DEVELOPMENT_QUERIES_PER_MONTH,
    "moderate": MODERATE_QUERIES_PER_MONTH,
    "production": PRODUCTION_QUERIES_PER_MONTH
}

queries_per_month = query_counts.get(QUERY_SCENARIO, DEVELOPMENT_QUERIES_PER_MONTH)

# Estimate vectors processed per query
# HNSW typically examines 100-500 vectors for topK=10
# With metadata filters, candidate set is pre-filtered, so similar range
AVG_VECTORS_PROCESSED_PER_QUERY = 300  # Conservative estimate

total_vectors_processed = queries_per_month * AVG_VECTORS_PROCESSED_PER_QUERY

# API call cost
query_api_cost = (queries_per_month / 1_000_000) * QUERY_API_COST_PER_MILLION

# Data processing cost (tiered)
query_processing_tb = (total_vectors_processed * query_processing_size_per_vector) / (1024 ** 4)

# Simplified: Use tier 1 rate for small volumes
query_data_cost = query_processing_tb * QUERY_DATA_COST_TIER1

total_query_cost = query_api_cost + query_data_cost

print(f"\n{'='*70}")
print(f"COST 3: QUERY COSTS - {QUERY_SCENARIO.upper()} SCENARIO")
print(f"{'='*70}")
print(f"  Queries per month: {queries_per_month:,}")
print(f"  Avg vectors processed per query: {AVG_VECTORS_PROCESSED_PER_QUERY}")
print(f"  Total vectors processed: {total_vectors_processed:,}")

print(f"\n  [API Call Cost]")
print(f"    Rate: ${QUERY_API_COST_PER_MILLION:.2f} per million calls")
print(f"    Cost: ${query_api_cost:.4f}")

print(f"\n  [Data Processing Cost]")
print(f"    Query processing size per vector: {query_processing_size_per_vector/1024:.2f} KB")
print(f"    Total data processed: {query_processing_tb:.6f} TB")
print(f"    Rate: ${QUERY_DATA_COST_TIER1:.3f} per TB (tier 1)")
print(f"    Cost: ${query_data_cost:.4f}")

print(f"\n  Monthly query cost: ${total_query_cost:.4f}")

# ============================================================================
# TOTAL COST SUMMARY
# ============================================================================

# First month (includes upload)
first_month_total = upload_cost + monthly_storage_cost + total_query_cost

# Ongoing monthly (no upload)
ongoing_monthly = monthly_storage_cost + total_query_cost

print(f"\n{'='*70}")
print(f"💰 TOTAL COST SUMMARY - {QUERY_SCENARIO.upper()}")
print(f"{'='*70}")

print(f"\n[First Month - Includes Upload]")
print(f"  Upload (one-time): ${upload_cost:.4f}")
print(f"  Storage: ${monthly_storage_cost:.4f}")
print(f"  Queries: ${total_query_cost:.4f}")
print(f"  ─────────────────────────")
print(f"  Total: ${first_month_total:.2f}")

print(f"\n[Ongoing Monthly Costs]")
print(f"  Storage: ${monthly_storage_cost:.4f}")
print(f"  Queries: ${total_query_cost:.4f}")
print(f"  ─────────────────────────")
print(f"  Total: ${ongoing_monthly:.2f}/month")
print(f"  Annual: ${ongoing_monthly * 12:.2f}/year")

# ============================================================================
# COMPARISON: ALL SCENARIOS
# ============================================================================

print(f"\n{'='*70}")
print(f"COST COMPARISON: QUERY SCENARIOS")
print(f"{'='*70}")

for scenario_name, query_count in query_counts.items():
    scenario_vectors_processed = query_count * AVG_VECTORS_PROCESSED_PER_QUERY
    scenario_api_cost = (query_count / 1_000_000) * QUERY_API_COST_PER_MILLION
    scenario_processing_tb = (scenario_vectors_processed * query_processing_size_per_vector) / (1024 ** 4)
    scenario_data_cost = scenario_processing_tb * QUERY_DATA_COST_TIER1
    scenario_query_cost = scenario_api_cost + scenario_data_cost
    scenario_monthly = monthly_storage_cost + scenario_query_cost
    
    print(f"\n{scenario_name.capitalize()} ({query_count:,} queries/month):")
    print(f"  Storage: ${monthly_storage_cost:.4f}")
    print(f"  Queries: ${scenario_query_cost:.4f}")
    print(f"  Total: ${scenario_monthly:.2f}/month (${scenario_monthly * 12:.2f}/year)")

print(f"\n{'='*70}")
print(f"✓ COST ESTIMATION COMPLETE")
print(f"{'='*70}")
print(f"\nKey takeaways:")
print(f"  • Upload is one-time: ${upload_cost:.4f}")
print(f"  • Storage is cheap: ${monthly_storage_cost:.4f}/month")
print(f"  • Query costs scale with usage")
print(f"  • Non-filterable metadata doesn't affect query costs")
print(f"  • Current scenario: ~${ongoing_monthly:.2f}/month total")
print("="*70)


[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
S3 VECTORS COST ESTIMATION
Provider: cohere_1024d
Data source: finrag_embeddings_s3vectors_cohere_1024d.parquet

[Vector Index Statistics]
  Total vectors: 203,076
  Dimension: 1024d

[Size Breakdown per Vector]
  Vector data: 4,096 bytes (4.00 KB)
  Key: 20 bytes
  Filterable metadata: 19 bytes
  Non-filterable metadata: 75 bytes
  Total per vector: 4,210 bytes (4.11 KB)

[Total Logical Size]
  Storage size: 0.7962 GB (815.34 MB)
  Upload size: 0.7962 GB

COST 1: ONE-TIME UPLOAD (PutVectors)
  Data uploaded: 0.7962 GB
  Rate: $0.20 per GB
  Total upload cost: $0.1592
  Status: Pay once when inserting vectors

COST 2: MONTHLY STORAGE
  Data stored: 0.7962 GB
  Rate: $0.06 per GB-month
  Monthly storage cost: $0.0478
  Annual storage cost: $0.57

COST 3: QUERY COSTS - DEVELOPMENT SCENARIO
  Queries per month: 1,000
  Avg vectors processed per query: 300
  Total vectors processed: 300,000

  [API Call Cost]
    Rate: $2.50 per mil